In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import Packages.ClusteringHelper as ch
from Packages.TimeEvolving import DataEvolver
from textdistance import DamerauLevenshtein, Levenshtein
import numpy as np
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from Packages.TimeEvolving import Cluster
from tqdm.notebook import tqdm
import math
import sys
from collections import Counter
import time
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import pairwise_distances

In [4]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
ents_data = data[data['entities'] != ''].copy()

In [5]:
ents_data = ch.add_entities_embedding(ents_data,
                                      "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings")
ents_data_filtered = ents_data.copy()
documents = set(ents_data.documents)

In [6]:
evolving = DataEvolver(documents, ents_data, randomly=False, step=10)
gold_entities = []
total_clusters = []
n=0

## Let the cycle start


In [27]:
tic = time.perf_counter()
for iteration in tqdm(evolving, total=math.ceil(len(evolving.documents) / evolving.step)):
    current_mentions = list(evolving.get_current_data().mentions)
    current_encodings = list(evolving.get_current_data()['encodings'].values)
    current_entities = list(evolving.get_current_data()['entities'].values)
    def lev_metric(x, y):
        i, j = int(x[0]), int(y[0])  # extract indices
        if len(current_mentions[i]) < 4:
            if current_mentions[i] == current_mentions[j]:
                return 0
            else:
                return Levenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower()) + 3
        else:
            return Levenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower())
    def dam_lev_metric(x, y):
        i, j = int(x[0]), int(y[0])  # extract indices
        if len(current_mentions[i]) < 4:
            if current_mentions[i] == current_mentions[j]:
                return 0
            else:
                return DamerauLevenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower()) + 3
        else:
            return DamerauLevenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower())
    X = np.arange(len(current_mentions)).reshape(-1, 1)
    m_matrix = pairwise_distances(X, X, metric=dam_lev_metric, n_jobs=-1)
    # clusterizator1 = DBSCAN(metric=dam_lev_metric, eps=1, min_samples=0, n_jobs=-1)
    clusterizator1 = AgglomerativeClustering(n_clusters=None, affinity='precomputed',
                                             distance_threshold=1,
                                             linkage="single")
    cluster_numbers = clusterizator1.fit_predict(m_matrix)
    cee_dict = {k: {'entities': [], 'mentions': [], 'encodings': [], 'sotto_clusters': None} for k in
                set(cluster_numbers)}
    for i, cluster in enumerate(cluster_numbers):
        cee_dict[cluster]['entities'].append(current_entities[i])
        cee_dict[cluster]['mentions'].append(current_mentions[i])
        cee_dict[cluster]['encodings'].append(current_encodings[i])
    cee_list = cee_dict.values()
    clusterizator2 = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=first_threshold,
                                             linkage="single")
    for cluster in cee_dict.keys():
        try:
            cee_dict[cluster]['sotto_clusters'] = clusterizator2.fit_predict(cee_dict[cluster]['encodings'])
        except ValueError:
            cee_dict[cluster]['sotto_clusters'] = np.zeros(1, dtype=np.int8)
    sottocluster_list = []
    for el in cee_list:
        sotto_cluster = {k: Cluster() for k in set(el['sotto_clusters'])}
        for i, key in enumerate(el['sotto_clusters']):
            sotto_cluster[key].add_element(mention=el['mentions'][i], entity=el['entities'][i],
                                           encodings=el['encodings'][i])
        sottocluster_list.append(sotto_cluster)
    sottocluster_list = [clusters_dict[key] for clusters_dict in sottocluster_list for key in clusters_dict]
    current_clusters = total_clusters + sottocluster_list
    sotto_encodings = [x.encodings_mean for x in current_clusters]
    clusterizator3 = AgglomerativeClustering(n_clusters=None, affinity='cosine',
                                             distance_threshold=second_threshold,
                                             linkage="single")
    cluster_numbers = clusterizator3.fit_predict(sotto_encodings)
    final_clusters = {k: Cluster() for k in set(cluster_numbers)}
    for i, x in enumerate(current_clusters):
        try:
            final_clusters[cluster_numbers[i]] = final_clusters[cluster_numbers[i]] + x
        except:
            print(cluster_numbers[i], final_clusters[cluster_numbers[i]], x)
    gold_entities = gold_entities + current_entities
    total_clusters = list(final_clusters.values())
    total_clusters = [x for x in total_clusters if len(set(x.mentions)) < 15]
    # CEAFm
    best_alignment = ch.get_optimal_alignment([x.count_ents for x in total_clusters], set(gold_entities),
                                              is_dict=False)
    CEAFm_p = sum(best_alignment.values()) / len(gold_entities)
    CEAFm_r = sum(best_alignment.values()) / sum([x.n_elements for x in total_clusters])
    CEAFm_f1 = 2 * (CEAFm_p * CEAFm_r) / (CEAFm_p + CEAFm_r)
    original_stdout = sys.stdout
    # with open(".\\Results\\step" + str(n) + ".txt", "a") as f:
    print("CEAFm-R:", CEAFm_r,'<br>')
    print("CEAFm-P:", CEAFm_p,'<br>')
    print("CEAFm:", CEAFm_f1,'<br>')
    print("Clusters:")
    print(*total_clusters, sep=" <br><br> ")
    # print("Gold_standard:")
    # print(Counter(gold_entities))
    n = n +1
    break
toc = time.perf_counter()
print(toc - tic)

  0%|          | 0/139 [00:00<?, ?it/s]

CEAFm-R: 0.9003436426116839 <br>
CEAFm-P: 0.9003436426116839 <br>
CEAFm: 0.900343642611684 <br>
Clusters:
Cluster{'<b>Washington,_D.C.</b>': {'WASHINGTON': 3, 'Washington': 3, '<span style="background: yellow;">#</span>': 6}}; <span style="background: pink;">#_elements</span> = 6 <br><br> Cluster{'<b>Nepal</b>': {'Nepal': 4, '<span style="background: yellow;">#</span>': 4}}; <span style="background: pink;">#_elements</span> = 4 <br><br> Cluster{'<b>United_States</b>': {'United States': 5, 'U.S.': 11, '<span style="background: yellow;">#</span>': 16}}; <span style="background: pink;">#_elements</span> = 16 <br><br> Cluster{'<b>Iraq</b>': {'Iraq': 9, 'Iraqi': 8, '<span style="background: yellow;">#</span>': 17}}; <span style="background: pink;">#_elements</span> = 17 <br><br> Cluster{'<b>Reuters</b>': {'Reuters': 4, '<span style="background: yellow;">#</span>': 4}}; <span style="background: pink;">#_elements</span> = 4 <br><br> Cluster{'<b>Kurdish_people</b>': {'Kurdish': 1, '<span style